In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `~/.julia/dev/SymArrays/examples`


In [2]:
using Printf
using BenchmarkTools
using SymArrays
using SymArrays: SymIndexIter
println("Julia version $VERSION")

Julia version 1.7.0


┌ Info: Precompiling SymArrays [c2b21f84-4a8e-4c96-99a2-2b79df568f75]
└ @ Base loading.jl:1423


In [3]:
sumfirstinds(iter) = (s=0; for I in iter s+=first(Tuple(I)) end; s)
sumvals(iter) = reduce(+, iter);

In [4]:
ttt(iter) = (s=0; for ii in 1:length(iter) s+=first(SymArrays.ind2sub_symgrp(iter,ii)) end; s);

In [5]:
@btime ttt(SI)          setup=(SI=SymIndexIter(4,50))

  13.715 ms (0 allocations: 0 bytes)


3162510

In [6]:
@btime sumfirstinds(SI) setup=(SI=SymIndexIter(4,50))

  221.104 μs (0 allocations: 0 bytes)


3162510

In [7]:
for n in (10, 50, 200)
    n != 10 && println()
    println("n = $n:")
    for N in 1:4, M in (N,-N)
        iter = SymIndexIter(M,n)
        # check that number of elements is correct
        @assert count(x->true,iter) == length(iter)

        @printf("  M = %2d:", M)
        @btime sumfirstinds($iter)
    end
end

n = 10:
  M =  1:  2.253 ns (0 allocations: 0 bytes)
  M = -1:  2.251 ns (0 allocations: 0 bytes)
  M =  2:  48.586 ns (0 allocations: 0 bytes)
  M = -2:  37.539 ns (0 allocations: 0 bytes)
  M =  3:  226.823 ns (0 allocations: 0 bytes)
  M = -3:  125.763 ns (0 allocations: 0 bytes)
  M =  4:  857.175 ns (0 allocations: 0 bytes)
  M = -4:  262.569 ns (0 allocations: 0 bytes)

n = 50:
  M =  1:  1.819 ns (0 allocations: 0 bytes)
  M = -1:  2.252 ns (0 allocations: 0 bytes)
  M =  2:  913.658 ns (0 allocations: 0 bytes)
  M = -2:  844.234 ns (0 allocations: 0 bytes)
  M =  3:  15.335 μs (0 allocations: 0 bytes)
  M = -3:  15.591 μs (0 allocations: 0 bytes)
  M =  4:  205.859 μs (0 allocations: 0 bytes)
  M = -4:  147.769 μs (0 allocations: 0 bytes)

n = 200:
  M =  1:  2.251 ns (0 allocations: 0 bytes)
  M = -1:  1.824 ns (0 allocations: 0 bytes)
  M =  2:  9.943 μs (0 allocations: 0 bytes)
  M = -2:  23.059 μs (0 allocations: 0 bytes)
  M =  3:  734.565 μs (0 allocations: 0 bytes)
  M =

In [8]:
for S in (SymArray{(3,),Float64}(50,50,50),
          SymArray{(-3,),Float64}(50,50,50),
          SymArray{(3,-2,2),Float64}(2,2,2,3,3,4,4),
          SymArray{(3,2,2),Float64}(2,2,2,3,3,4,4),
          SymArray{(-3,-4),Float64}(8,8,8,5,5,5,5),
          SymArray{(3,-2,2),Float64}(2,2,2,30,30,4,4),
          SymArray{(3,2,2),Float64}(2,2,2,30,30,4,4),
          SymArray{(-3,-4),Float64}(20,20,20,5,5,5,5))
    @printf("typeof(S): %-54s, size(S): %s, length(S): %d, full length(S): %d\n", typeof(S), size(S), length(S), prod(size(S)))
    fill!(S,1.)
    @btime sumfirstinds($(eachindex(S)))
    @btime sumvals($S)
end

typeof(S): SymArray{(3,), Float64, 3, 1, Vector{Float64}}        , size(S): (50, 50, 50), length(S): 22100, full length(S): 125000
  13.430 μs (0 allocations: 0 bytes)
  148.917 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(-3,), Float64, 3, 1, Vector{Float64}}       , size(S): (50, 50, 50), length(S): 19600, full length(S): 125000
  12.976 μs (0 allocations: 0 bytes)
  154.704 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(3, -2, 2), Float64, 7, 3, Array{Float64, 3}}, size(S): (2, 2, 2, 3, 3, 4, 4), length(S): 120, full length(S): 1152
  264.473 ns (0 allocations: 0 bytes)
  1.746 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(3, 2, 2), Float64, 7, 3, Array{Float64, 3}} , size(S): (2, 2, 2, 3, 3, 4, 4), length(S): 240, full length(S): 1152
  541.593 ns (0 allocations: 0 bytes)
  2.543 μs (0 allocations: 0 bytes)
typeof(S): SymArray{(-3, -4), Float64, 7, 2, Matrix{Float64}}    , size(S): (8, 8, 8, 5, 5, 5, 5), length(S): 280, full length(S): 320000
  324.764 ns (0 allocations: 0